In [2]:
import os

In [7]:
from dotenv import load_dotenv
import os
import boto3

load_dotenv()

s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_DEFAULT_REGION")
)

s3_bucket_name = os.getenv("AWS_S3_BUCKET_NAME")


In [2]:
def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)

    try:
        s3_client.upload_file(file_name, bucket, object_name)
        return True
    except Exception as e:
        print(e)
        return False


In [4]:
def list_bucket_files(bucket):
    resp = s3_client.list_objects_v2(Bucket=bucket)
    if "Contents" not in resp:
        return []

    return [item["Key"] for item in resp["Contents"]]

In [ ]:
for file_name in os.listdir("../data"):
    file_path = f"../data/{file_name}"
    # print(file_path)
    upload_file(file_path, s3_bucket_name)
    

In [12]:
for file_name in list_bucket_files(s3_bucket_name):
    s3_client.download_file(s3_bucket_name, file_name, f'../data_from_s3/{file_name}')

In [ ]:
s3 = boto3.resource("s3")
bucket = s3.Bucket(s3_bucket_name)

for obj in bucket.objects.all():
    obj.delete()